Grabs the top 5 names from the fraudTest.csv file.

In [2]:
import pandas as pd

# Load the CSV file. Adjust the filename/path as needed.
df = pd.read_csv('../fraudTest.csv', index_col=0)

# Group by the first and last name, and count transactions for each group.
transaction_counts = df.groupby(['first', 'last']).size().reset_index(name='transaction_count')

# Sort the result in descending order by the transaction count.
top_names = transaction_counts.sort_values('transaction_count', ascending=False)

print(top_names.head(5))


        first        last  transaction_count
788     Scott      Martin               1965
385   Jeffrey       Smith               1526
314      Gina      Grimes               1474
655  Michelle     Gregory               1466
132    Carrie  Washington               1462


In [ ]:
# Convert the top 5 names to a list of tuples for easier iteration
top_5_names = top_names.head(5)[['first', 'last']].values.tolist()

transactions_by_purchaser = {}

for first, last in top_5_names:
    # Filter the original DataFrame for each purchaser
    transactions = df[(df['first'] == first) & (df['last'] == last)]
    transactions_by_purchaser[(first, last)] = transactions
    print(f"Transactions for {first} {last}:")
    print(transactions)


Transactions for Scott Martin:
       trans_date_trans_time            cc_num  \
407      2020-06-21 14:31:28  3502088871723054   
517      2020-06-21 15:08:54  3502088871723054   
770      2020-06-21 16:36:37  3502088871723054   
962      2020-06-21 17:43:07  4334230547694630   
1704     2020-06-21 22:13:27  3502088871723054   
...                      ...               ...   
554914   2020-12-31 19:15:35  4334230547694630   
554924   2020-12-31 19:16:58  4334230547694630   
555016   2020-12-31 19:46:44  3502088871723054   
555029   2020-12-31 19:54:46  4334230547694630   
555401   2020-12-31 22:04:19  3502088871723054   

                                  merchant       category    amt  first  \
407               fraud_Altenwerth-Kilback           home  27.12  Scott   
517                      fraud_Osinski Inc  personal_care  23.33  Scott   
770                    fraud_Hills-Witting   shopping_net  59.46  Scott   
962     fraud_Gutmann, McLaughlin and Wiza           home   9.62  Sc